## Import Libraries

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

## Loading and cleaning the data

In [17]:
data = pd.read_csv("/content/Churn_Modelling.csv")
data = data.drop(columns = ["RowNumber","CustomerId", "Surname"]) # dropping columns which don't serve a meaningful purpose for prediction
label_encoder_X = LabelEncoder()
data["Geography"] = label_encoder_X.fit_transform(data["Geography"])  # Encoding 'Geography'
data["Gender"] = label_encoder_X.fit_transform(data["Gender"])  # Encoding 'Gender'
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0.00,2,1,0,96270.64,0
9996,516,0,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,0,0,36,7,0.00,1,0,1,42085.58,1
9998,772,1,1,42,3,75075.31,2,1,0,92888.52,1


# Extract features and target variable

In [18]:
X_pd = data.iloc[:, :-1]
y_pd = data.iloc[:, -1]

In [19]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Splitting and Scaling

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Building the Artificial Neural Network


In [21]:
classifier = Sequential()

# Add the input layer
classifier.add(Dense(units=X_pd.shape[1], kernel_initializer='uniform', activation='relu', input_dim=X_pd.shape[1]))

# Add hidden layers and dropout
classifier.add(Dense(units=5, kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate=0.1))
classifier.add(Dense(units=3, kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate=0.1))

classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid')) # Output layer



# Compiling the ANN model using different optimizers

In [22]:
# Compile the ANN
model_performances = {}
optimizers_list = ['sgd', 'rmsprop', 'adagrad', 'adadelta', 'adam', 'nadam']
for optimizer in optimizers_list:
  classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  # Define early stopping callback
  early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

  # Fit the ANN to the training set
  classifier.fit(X_train, y_train, batch_size=10, epochs=100, validation_split=0.1, callbacks=[early_stopping])

  # Predict the test set results
  y_pred = classifier.predict(X_test)
  y_pred = (y_pred > 0.5)

  # Evaluate the performance
  accuracy = accuracy_score(y_test, y_pred)
  conf_matrix = confusion_matrix(y_test, y_pred)

  result = {"accuracy": accuracy, "confusion_matrix": conf_matrix}
  model_performances[optimizer] = result


Epoch 1/100
630/630 [==============================] - 2s 2ms/step - loss: 0.5731 - accuracy: 0.7914 - val_loss: 0.5224 - val_accuracy: 0.7943
Epoch 2/100
630/630 [==============================] - 1s 2ms/step - loss: 0.5164 - accuracy: 0.7922 - val_loss: 0.5100 - val_accuracy: 0.7943
Epoch 3/100
630/630 [==============================] - 2s 3ms/step - loss: 0.5117 - accuracy: 0.7922 - val_loss: 0.5085 - val_accuracy: 0.7943
Epoch 4/100
630/630 [==============================] - 2s 3ms/step - loss: 0.5112 - accuracy: 0.7922 - val_loss: 0.5083 - val_accuracy: 0.7943
Epoch 5/100
630/630 [==============================] - 1s 2ms/step - loss: 0.5111 - accuracy: 0.7922 - val_loss: 0.5082 - val_accuracy: 0.7943
Epoch 6/100
630/630 [==============================] - 1s 2ms/step - loss: 0.5111 - accuracy: 0.7922 - val_loss: 0.5082 - val_accuracy: 0.7943
Epoch 7/100
630/630 [==============================] - 1s 2ms/step - loss: 0.5111 - accuracy: 0.7922 - val_loss: 0.5082 - val_accuracy: 0.7943

# Final results

In [ ]:
for optimizer, result in model_performances.items():
  print(optimizer)
  print(f"Accuracy: {result['accuracy']*100}%")
  print(f"Confusion Matrix:\n{result['confusion_matrix']}")
  print()